<a href="https://colab.research.google.com/github/MEHARKhaoula/transformers-text-coherence/blob/main/BERTSyn2_concat_yahoo_scenario2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-88ac7f3e-e75c-b8d8-700b-ebb2697fc50f)


In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


# 1. Setup

## 1.1. Using Colab GPU for Training


In [3]:
import tensorflow as tf
import pandas as pd
# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


## 1.2. Installing dependencies


In order for torch to use the GPU, we need to identify and specify the GPU as the device. Later, in our training loop, we will load data onto the device. 

In [4]:
!pip install transformers
!pip install jsonlines

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 67.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 93.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import torch
import spacy
# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


# Importing GCDC dataset



## 2.2. Parse

We'll use pandas to parse the "in-domain" training set and look at a few of its properties and data points.

In [6]:

# Load the dataset for training into a pandas dataframe.
df = pd.read_csv("/content/drive/My Drive/GCDC_train.csv")
df['labelA'] = df['labelA'].astype(int)
df['labelA'] = df['labelA'] - 1

df_test = pd.read_csv("/content/drive/My Drive/Yahoo_test.csv")
df_test['labelA'] = df_test['labelA'].astype(int)
df_test['labelA'] = df_test['labelA'] - 1

# Report the number of sentences.
print('Number of training docs: {:,}\n'.format(df.shape[0]))
print('Number of training docs: {:,}\n'.format(df_test.shape[0]))
# Display 10 random rows from the data.
df.sample(10)
     


Number of training docs: 4,000

Number of training docs: 200



,text_id,subject,text,ratingA1,ratingA2,ratingA3,labelA,ratingM1,ratingM2,ratingM3,ratingM4,ratingM5,labelM
2983,1285127,NaN,will do.\n\nThe purpose of nymex on weekends i...,2,1,1,0,2,1,2,1,2,1
2236,1113061,RTO Question,"On this same subject of Chairman Wood, I am to...",1,2,1,0,1,2,3,1,2,1
3965,text563NE,NaN,And this is surprising how? The news media is...,1,2,2,0,2,2,2,2,1,1
3508,text798NE,NaN,National Association of Securities Dealers Aut...,3,2,3,2,2,3,3,3,2,3
1887,N4V6JpV3xyNWt0paJOaKWA,Nora Shawarma,The shawarma at this place is great. But just ...,1,1,2,0,2,2,3,2,1,2
2126,1112173,Midwest Regional Budget,Rick after the joint Midwest-Atlantic Meeting ...,3,2,2,2,3,2,2,1,1,1
1854,XmDJfgC9RfPXXdmZzB05yw,The Federal,This is a brunch spot I would gladly come back...,1,2,2,0,2,1,1,1,2,1
1846,xd4ZW9UBlrRLunIhOINwYg,Diamond China 2,I grew up going to the Meadows School which is...,3,2,2,2,2,2,2,2,3,2
825,C05791065_3,Post 2015 call follow up,"Cheryl, At the Rio+20 conference USUN/NY's Eli...",2,2,2,1,1,2,1,2,2,1
3592,text1303PG,NaN,Disagree and debate all you like. Debate is ac...,2,3,3,2,2,3,3,3,2,3


In [7]:
# Get the lists of docs and their labels.
documents = df.text.values
labels = df.labelA.values

# Get the lists of test docs and their labels.
documents_test = df_test.text.values
labels_test = df_test.labelA.values
     

The two properties we actually care about are the the `text` and its `labelA`, which is referred to as the "coherence class"



Let's extract the sentences and labels of our training set as numpy ndarrays.

# 3. Tokenization & Input Formatting

In this section, we'll transform our dataset into the format that BERT can be trained on.

## 3.1. BERT Tokenizer

In [8]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer...


Let's apply the tokenizer to one sentence just to see the output.


In [9]:
'''
# Print the original sentence.
print(' Original: ', documents[0])

# Print the sentence split into tokens.
print('Tokenized: ', tokenizer.tokenize(documents[0]))

# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(documents[0])))
'''

"\n# Print the original sentence.\nprint(' Original: ', documents[0])\n\n# Print the sentence split into tokens.\nprint('Tokenized: ', tokenizer.tokenize(documents[0]))\n\n# Print the sentence mapped to token ids.\nprint('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(documents[0])))\n"

## 3.2. Required Formatting

### Special Tokens


## 3.3. Tokenize Dataset

In [10]:
'''
max_len = 0

# For every sentence...
for doc in documents:

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(doc, add_special_tokens=True)

    # Update the maximum sentence length.
    max_len = max(max_len, len(input_ids))

# For every sentence...
for doc in documents_test:

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids_test = tokenizer.encode(doc, add_special_tokens=True)

    # Update the maximum sentence length.
       

print('Max sentence length: ', max_len)

'''

"\nmax_len = 0\n\n# For every sentence...\nfor doc in documents:\n\n    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.\n    input_ids = tokenizer.encode(doc, add_special_tokens=True)\n\n    # Update the maximum sentence length.\n    max_len = max(max_len, len(input_ids))\n\n# For every sentence...\nfor doc in documents_test:\n\n    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.\n    input_ids_test = tokenizer.encode(doc, add_special_tokens=True)\n\n    # Update the maximum sentence length.\n       \n\nprint('Max sentence length: ', max_len)\n\n"

Now we're ready to perform the real tokenization.

The `tokenizer.encode_plus` function combines multiple steps for us:

1. Split the sentence into tokens.
2. Add the special `[CLS]` and `[SEP]` tokens.
3. Map the tokens to their IDs.
4. Pad or truncate all sentences to the same length.
5. Create the attention masks which explicitly differentiate real tokens from `[PAD]` tokens.

The first four features are in `tokenizer.encode`, but I'm using `tokenizer.encode_plus` to get the fifth item (attention masks). Documentation is [here](https://huggingface.co/transformers/main_classes/tokenizer.html?highlight=encode_plus#transformers.PreTrainedTokenizer.encode_plus).


In [11]:
    import torch
    
    


   
# For every sentence...
    """
    def preprocessing(documents,labels,pos_tags):
    
        input_ids_list = []
        attention_masks = []
        input_ids2_list = []
        attention_masks2 = []
        
        for doc , pos_tag in zip(documents,pos_tags):
            # `encode_plus` will:
            #   (1) Tokenize the sentence.
            #   (2) Prepend the `[CLS]` token to the start.
            #   (3) Append the `[SEP]` token to the end.
            #   (4) Map tokens to their IDs.
            #   (5) Pad or truncate the sentence to `max_length`
            #   (6) Create attention masks for [PAD] tokens.
            input_ids = []
            
            input_ids = tokenizer.encode(' '.join(doc), add_special_tokens=True)
              # Append the input IDs to the list
            
            if len(input_ids) > 200:
                  input_ids = input_ids[: 200]
            attention_mask = [1] * len(input_ids)
            # padding up to max_seq_length
            padding_length = 200- len(input_ids)
            input_ids += [tokenizer.pad_token_id] * padding_length
            attention_mask += [0] * padding_length
            
            input_ids= torch.tensor(input_ids).unsqueeze(0)   
            print(input_ids)
            attention_mask=torch.tensor(attention_mask).unsqueeze(0)
            input_ids_list.append(input_ids)
            attention_masks.append(attention_mask)

            input_ids = []
            
            input_ids = tokenizer.encode(' '.join(pos_tag), add_special_tokens=True)
              # Append the input IDs to the list
            
            if len(input_ids) > 200:
                  input_ids = input_ids[: 200]
            attention_mask = [1] * len(input_ids)
            # padding up to max_seq_length
            padding_length = 200- len(input_ids)
            input_ids += [tokenizer.pad_token_id] * padding_length
            attention_mask += [0] * padding_length
            
            input_ids= torch.tensor(input_ids).unsqueeze(0)   
            print(input_ids)
            attention_mask=torch.tensor(attention_mask).unsqueeze(0)
            input_ids2_list.append(input_ids)
            attention_masks2.append(attention_mask)
            
           
            
          

          

            

        # Convert the lists into tensors.
        input_ids_list = torch.cat(input_ids_list, dim=0)
        attention_masks = torch.cat(attention_masks, dim=0)
        input_ids2_list = torch.cat(input_ids2_list, dim=0)
        attention_masks2 = torch.cat(attention_masks2, dim=0)
        labels = torch.tensor(labels)

        return input_ids_list, attention_masks , input_ids2_list , attention_masks2 , labels
        
    input_ids, attention_masks , input_ids2 , attention_masks2 , labels = preprocessing(documents,labels,pos_tags)
    input_ids_test, attention_masks_test , input_ids2_test , attention_masks2_test , labels_test = preprocessing(documents_test,labels_test,pos_tags_test)
"""
    import torch
    nlp = spacy.load("en_core_web_sm")
    # Tokenize all of the sentences and map the tokens to thier word IDs.


   
# For every sentence...
    
    def preprocessing(documents,labels):
    
        input_ids = []
        attention_masks = []
        input_ids2 = []
        attention_masks2 = []
        
        for doc in documents:
            # `encode_plus` will:
            #   (1) Tokenize the sentence.
            #   (2) Prepend the `[CLS]` token to the start.
            #   (3) Append the `[SEP]` token to the end.
            #   (4) Map tokens to their IDs.
            #   (5) Pad or truncate the sentence to `max_length`
            #   (6) Create attention masks for [PAD] tokens.
            tagging = nlp(doc)
            pos_tags = [token.pos_ for token in tagging]
            
            pos_tag_ids = [tagging.vocab.strings[tag] for tag in pos_tags]
           
            encoded_dict = tokenizer.encode_plus(
                                doc,                      # Sentence to encode.
                                add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                                max_length = 256,           # Pad & truncate all sentences.
                                pad_to_max_length = True,
                                return_attention_mask = True,   # Construct attn. masks.
                                return_tensors = 'pt',     # Return pytorch tensors.
                          )
            
            encoded_dict2 = tokenizer.encode_plus(
                                ' '.join(pos_tags),                      # Sentence to encode.
                                add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                                max_length = 256,           # Pad & truncate all sentences.
                                pad_to_max_length = True,
                                return_attention_mask = True,   # Construct attn. masks.
                                return_tensors = 'pt',     # Return pytorch tensors.
                          )
          

            # Add the encoded sentence to the list.    
            input_ids.append(encoded_dict['input_ids'])
            
            # And its attention mask (simply differentiates padding from non-padding).
            attention_masks.append(encoded_dict['attention_mask'])
            
            input_ids2.append(encoded_dict2['input_ids'])
            
            # And its attention mask (simply differentiates padding from non-padding).
            attention_masks2.append(encoded_dict2['attention_mask'])
          

        # Convert the lists into tensors.
        input_ids = torch.cat(input_ids, dim=0)
        attention_masks = torch.cat(attention_masks, dim=0)
        input_ids2 = torch.cat(input_ids2, dim=0)
        attention_masks2 = torch.cat(attention_masks2, dim=0)
        labels = torch.tensor(labels)

        return input_ids, attention_masks , input_ids2 , attention_masks2 , labels
        
    input_ids, attention_masks , input_ids2 , attention_masks2 , labels = preprocessing(documents,labels)
    input_ids_test, attention_masks_test , input_ids2_test , attention_masks2_test , labels_test = preprocessing(documents_test,labels_test)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2377: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [12]:
from torch.utils.data import TensorDataset, random_split

# Combine the training inputs into a TensorDataset.
train_dataset = TensorDataset(input_ids,attention_masks,input_ids2,attention_masks2,labels)
test_dataset = TensorDataset(input_ids_test, 
                        attention_masks_test,input_ids2_test,attention_masks2_test,labels_test)


In [13]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 16

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )
test_dataloader = DataLoader(
            test_dataset,  # The training samples.
            sampler = RandomSampler(test_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )


# 4. Train Our Classification Model

Now that our input data is properly formatted, it's time to fine tune the BERT model. 

## 4.1. BertForSequenceClassification

In [14]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import json
from transformers import BertForSequenceClassification
from typing import List, Optional, Tuple, Union

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import json
from transformers import BertForSequenceClassification
from transformers import BertPreTrainedModel, BertConfig, BertModel



import math
import os
import warnings
from dataclasses import dataclass
from typing import Optional, Tuple

import torch
import torch.utils.checkpoint
from torch import nn
from torch.nn import CrossEntropyLoss, MSELoss

from transformers.activations import ACT2FN
from transformers.file_utils import (
    ModelOutput,
    add_code_sample_docstrings,
    add_start_docstrings,
    add_start_docstrings_to_model_forward,
    replace_return_docstrings,
)
from transformers.modeling_outputs import (
    BaseModelOutputWithCrossAttentions,
    BaseModelOutputWithPoolingAndCrossAttentions,
    CausalLMOutputWithCrossAttentions,
    MaskedLMOutput,
    MultipleChoiceModelOutput,
    NextSentencePredictorOutput,
    QuestionAnsweringModelOutput,
    SequenceClassifierOutput,
    TokenClassifierOutput,
)
from transformers.modeling_utils import (
    PreTrainedModel,
    apply_chunking_to_forward,
    find_pruneable_heads_and_indices,
    prune_linear_layer,
)
from transformers.utils import logging

from transformers import BertConfig



_CHECKPOINT_FOR_DOC = "bert-base-uncased"
_CONFIG_FOR_DOC = "BertConfig"
_TOKENIZER_FOR_DOC = "BertTokenizer"

_CHECKPOINT_FOR_SEQUENCE_CLASSIFICATION = "textattack/bert-base-uncased-yelp-polarity"
_SEQ_CLASS_EXPECTED_OUTPUT = "'LABEL_1'"
_SEQ_CLASS_EXPECTED_LOSS = 0.01
BERT_INPUTS_DOCSTRING = r"""
    Args:
        input_ids (`torch.LongTensor` of shape `({0})`):
            Indices of input sequence tokens in the vocabulary.
            Indices can be obtained using [`BertTokenizer`]. See [`PreTrainedTokenizer.encode`] and
            [`PreTrainedTokenizer.__call__`] for details.
            [What are input IDs?](../glossary#input-ids)
        attention_mask (`torch.FloatTensor` of shape `({0})`, *optional*):
            Mask to avoid performing attention on padding token indices. Mask values selected in `[0, 1]`:
            - 1 for tokens that are **not masked**,
            - 0 for tokens that are **masked**.
            [What are attention masks?](../glossary#attention-mask)
        token_type_ids (`torch.LongTensor` of shape `({0})`, *optional*):
            Segment token indices to indicate first and second portions of the inputs. Indices are selected in `[0,
            1]`:
            - 0 corresponds to a *sentence A* token,
            - 1 corresponds to a *sentence B* token.
            [What are token type IDs?](../glossary#token-type-ids)
        position_ids (`torch.LongTensor` of shape `({0})`, *optional*):
            Indices of positions of each input sequence tokens in the position embeddings. Selected in the range `[0,
            config.max_position_embeddings - 1]`.
            [What are position IDs?](../glossary#position-ids)
        head_mask (`torch.FloatTensor` of shape `(num_heads,)` or `(num_layers, num_heads)`, *optional*):
            Mask to nullify selected heads of the self-attention modules. Mask values selected in `[0, 1]`:
            - 1 indicates the head is **not masked**,
            - 0 indicates the head is **masked**.
        inputs_embeds (`torch.FloatTensor` of shape `({0}, hidden_size)`, *optional*):
            Optionally, instead of passing `input_ids` you can choose to directly pass an embedded representation. This
            is useful if you want more control over how to convert `input_ids` indices into associated vectors than the
            model's internal embedding lookup matrix.
        output_attentions (`bool`, *optional*):
            Whether or not to return the attentions tensors of all attention layers. See `attentions` under returned
            tensors for more detail.
        output_hidden_states (`bool`, *optional*):
            Whether or not to return the hidden states of all layers. See `hidden_states` under returned tensors for
            more detail.
        return_dict (`bool`, *optional*):
            Whether or not to return a [`~utils.ModelOutput`] instead of a plain tuple.
"""
class BertForSequenceClassification(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.config = config

        self.bert = BertModel(config)
        classifier_dropout = (
            config.classifier_dropout if config.classifier_dropout is not None else config.hidden_dropout_prob
        )
        self.dropout = nn.Dropout(classifier_dropout)
        self.classifier = nn.Linear(1536, config.num_labels)
        

        # Initialize weights and apply final processing
        self.post_init()

    @add_start_docstrings_to_model_forward(BERT_INPUTS_DOCSTRING.format("batch_size, sequence_length"))
    @add_code_sample_docstrings(
        processor_class=_TOKENIZER_FOR_DOC,
        checkpoint=_CHECKPOINT_FOR_SEQUENCE_CLASSIFICATION,
        output_type=SequenceClassifierOutput,
        config_class=_CONFIG_FOR_DOC,
        expected_output=_SEQ_CLASS_EXPECTED_OUTPUT,
        expected_loss=_SEQ_CLASS_EXPECTED_LOSS,
    )
    def forward(
        self,
        input_ids: Optional[torch.Tensor] = None,
        attention_mask: Optional[torch.Tensor] = None,
        input_ids2: Optional[torch.Tensor] = None,
        attention_mask2: Optional[torch.Tensor] = None,
        token_type_ids: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.Tensor] = None,
        head_mask: Optional[torch.Tensor] = None,
        inputs_embeds: Optional[torch.Tensor] = None,
        labels: Optional[torch.Tensor] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ) -> Union[Tuple[torch.Tensor], SequenceClassifierOutput]:
        r"""
        labels (`torch.LongTensor` of shape `(batch_size,)`, *optional*):
            Labels for computing the sequence classification/regression loss. Indices should be in `[0, ...,
            config.num_labels - 1]`. If `config.num_labels == 1` a regression loss is computed (Mean-Square loss), If
            `config.num_labels > 1` a classification loss is computed (Cross-Entropy).
        """
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        outputs1 = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        outputs2 = self.bert(
            input_ids=input_ids2,
            attention_mask=attention_mask2,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
          
        
        
       
        pooled_output1 = outputs1[1]
        pooled_output2 = outputs2[1]
        pooled_output = torch.cat([pooled_output1, pooled_output2] ,dim=-1)
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        loss = None
        if labels is not None:
            if self.config.problem_type is None:
                if self.num_labels == 1:
                    self.config.problem_type = "regression"
                elif self.num_labels > 1 and (labels.dtype == torch.long or labels.dtype == torch.int):
                    self.config.problem_type = "single_label_classification"
                else:
                    self.config.problem_type = "multi_label_classification"

            if self.config.problem_type == "regression":
                loss_fct = MSELoss()
                if self.num_labels == 1:
                    loss = loss_fct(logits.squeeze(), labels.squeeze())
                else:
                    loss = loss_fct(logits, labels)
            elif self.config.problem_type == "single_label_classification":
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            elif self.config.problem_type == "multi_label_classification":
                loss_fct = BCEWithLogitsLoss()
                loss = loss_fct(logits, labels)
        if not return_dict:
            output = (logits,) + outputs1[2:]
            return ((loss,) + output) if loss is not None else output

        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs1.hidden_states,
            attentions=outputs1.attentions,
        )


In [15]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import json
import torch
import spacy
from torch import nn
from transformers import BertTokenizer, BertModel



model = BertForSequenceClassification.from_pretrained(
          "bert-base-uncased", 
          num_labels = 3,  
          output_attentions = False,
          output_hidden_states = False,
      )

# Custom RNN model for learning POS tag sequences


# Custom model combining BERT and POSRNN

        



Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

In [16]:
from transformers import  AdamW, BertConfig

# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 

# Tell pytorch to run this model on the GPU.
model = model.to(device)

In [17]:
# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (30522, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (

## 4.2. Optimizer & Learning Rate Scheduler

In [18]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [19]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
epochs = 3

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

Helper function for formatting elapsed times as `hh:mm:ss`


In [20]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))


We're ready to kick off the training!

In [21]:
from sklearn.metrics import accuracy_score
from sklearn import metrics

def eval_docs(prediction_dataloader):
  # Prediction on test set
  # Put model in evaluation mode
  model.eval()

  # Tracking variables 
  predictions , true_labels = [], []

  # Predict 
  for batch in prediction_dataloader:
      # Add batch to GPU
      batch = tuple(t.to(device) for t in batch)
    
      # Unpack the inputs from our dataloader
      b_input_ids, b_input_mask,b_input_ids2, b_input_mask2,labels = batch
      
      # Telling the model not to compute or store gradients, saving memory and 
      # speeding up prediction
      with torch.no_grad():
          # Forward pass, calculate logit predictions.
         
          result = model(input_ids=b_input_ids, 
                     
                      attention_mask=b_input_mask, 
                      input_ids2=b_input_ids2, 
                      attention_mask2=b_input_mask2,
                      token_type_ids=None,

                      labels=labels
                        )
      results = F.softmax(result.logits, dim=0)

      logits = results
      

       

      # Move logits and labels to CPU
      logits = logits.detach().cpu().numpy()
      label_ids = labels.to('cpu').numpy()
      # Store predictions and true labels
      predictions.extend(list(np.argmax(logits, axis=1)))
      true_labels.extend(label_ids)

  matrix = metrics.confusion_matrix(true_labels, predictions, labels=[0, 1, 2])
  print(matrix)
  sum = np.sum(matrix)
  acc_low = (matrix[0][0] + matrix[1][1] + matrix[1][2] + matrix[2][1] + matrix [2][2])/(sum) #exactitude relative à la classe low
  acc_medium = (matrix[0][0] + matrix[1][1] + matrix[0][2] + matrix[2][0] + matrix [2][2])/(sum) #exactitude relative à la classe medium
  acc_high = (matrix[0][0] + matrix[1][1] + matrix[0][1] + matrix[1][0] + matrix [2][2])/(sum) #exactitude relative à la classe high

  accuracy = accuracy_score(predictions, true_labels)
  print("Accuracy :")
  print(accuracy)
  print("Accuracy low :")
  print(acc_low)  
  print("Accuracy medium :")
  print(acc_medium) 
  print("Accuracy high :")
  print(acc_high) 
  print("Average accuracy :")
  print((acc_low + acc_medium + acc_high)/3) 
  print("Classification report :")
  print(metrics.classification_report(true_labels, predictions, labels=[0, 1, 2], zero_division=1))

  print('    DONE.')
  return accuracy

In [22]:
import random
import numpy as np
import torch.nn.functional as F
import torch.utils.data as d
from torch.nn import CrossEntropyLoss, MSELoss
import sys


# Set the seed value all over the place to make this reproducible.
seed_val = 42 
best_test_acc = 0

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    CUDA_LAUNCH_BLOCKING=1
    # Put the model into training mode
    model.train()

    

     
    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

          # Progress update every 40 batches.
          if step % 40 == 0 and not step == 0:
              # Calculate elapsed time in minutes.
              elapsed = format_time(time.time() - t0)
              
              # Report progress.
              print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

          # Unpack this training batch from our dataloader. 
          batch = tuple(t.to(device) for t in batch)
    
          # Unpack the inputs from our dataloader
          b_input_ids, b_input_mask,b_input_ids2, b_input_mask2,labels = batch
          
          # Always clear any previously calculated gradients before performing a
          # backward pass.
          model.zero_grad()        

          # Perform a forward pass (evaluate the model on this training batch).

          result = model(
                        
                      input_ids=b_input_ids, 
                     
                      attention_mask=b_input_mask, 
                      input_ids2=b_input_ids2, 
                      attention_mask2=b_input_mask2,
                      labels=labels,
                      token_type_ids=None
                        )
          results = F.softmax(result.logits, dim=0)

          logits = results
          

          
          
         
          loss = result.loss

          # Accumulate the training loss over all of the batches so that we can
          # calculate the average loss at the end.
          total_train_loss += loss.item()
         
          # Perform a backward pass to calculate the gradients.
          loss.backward()

          torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

          # Update parameters and take a step using the computed gradient.
          optimizer.step()

          # Update the learning rate.
          scheduler.step()

      # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
      
      # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))

          # Record all statistics from this epoch.
    training_stats.append(
          {
              'epoch': epoch_i + 1,
              'Training Loss': avg_train_loss,
              'Training Time': training_time,
          }
      )
    test_accuracy = eval_docs(test_dataloader)
    if test_accuracy > best_test_acc:
      best_test_acc = test_accuracy


print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

print("==================== BEST TEST ACCURACY =================================")
print(best_test_acc)


======== Epoch 1 / 3 ========
Training...
  Batch    40  of    250.    Elapsed: 0:00:53.
  Batch    80  of    250.    Elapsed: 0:01:47.
  Batch   120  of    250.    Elapsed: 0:02:40.
  Batch   160  of    250.    Elapsed: 0:03:34.
  Batch   200  of    250.    Elapsed: 0:04:27.
  Batch   240  of    250.    Elapsed: 0:05:21.

  Average training loss: 0.95
  Training epcoh took: 0:05:34
[[42 21 19]
 [11 11 19]
 [ 8 13 56]]
Accuracy :
0.545
Accuracy low :
0.705
Accuracy medium :
0.68
Accuracy high :
0.705
Average accuracy :
0.6966666666666667
Classification report :
              precision    recall  f1-score   support

           0       0.69      0.51      0.59        82
           1       0.24      0.27      0.26        41
           2       0.60      0.73      0.65        77

    accuracy                           0.55       200
   macro avg       0.51      0.50      0.50       200
weighted avg       0.56      0.55      0.55       200

    DONE.

======== Epoch 2 / 3 ========
Training.

Let's view the summary of the training process.

In [23]:
import pandas as pd

# Display floats with two decimal places.
pd.set_option('precision', 2)

# Create a DataFrame from our training statistics.
df_stats = pd.DataFrame(data=training_stats)

# Use the 'epoch' as the row index.
df_stats = df_stats.set_index('epoch')

# A hack to force the column headers to wrap.
#df = df.style.set_table_styles([dict(selector="th",props=[('max-width', '70px')])])

# Display the table.
df_stats

OptionError: ignored

# Save the model

In [ ]:
import pickle

torch.save(model.state_dict(), './BERTSem.pt')
print('saved model ')

In [ ]:
# Fonction pour appliquer du preprocessing sur les données à fournir au modèle BERT
# avant de faire des prédictions (par prédiction je veux dire : donner un document en entrée et recevoir la classe prédite {Low, Medium, High})
# NOTE : il faut d'abord instancier le tokenizer + importer les librairies nécessaires (exécuter les cellules concernées dessus)
def preprocess_data_bert(text):
    input_ids = []
    attention_masks = []
    
    encoded_dict = tokenizer.encode_plus(
                        text,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 256,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])
    # Convert the lists into tensors.
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    
    sample = TensorDataset(input_ids, attention_masks)
    sample_loader = DataLoader(
            sample,
            sampler = RandomSampler(sample),
            batch_size = 1
        )
    return sample_loader

In [ ]:
model = torch.load(open('./BERTSem.pt', 'rb'), map_location=torch.device('0'))
model.eval()
sample_loader = preprocess_data_bert(sample)
for s in sample_loader: 
    sample = tuple(t for t in s)
b_input_ids, b_input_mask = sample
pred = model.forward(b_input_ids=b_input_ids, b_input_mask=b_input_mask)
result = F.softmax(pred.logits, dim=1)
result = result.cpu().data.numpy()
argmax = list(np.argmax(result, axis=1))
score = json.dumps(argmax[0], cls=NumpyArrayEncoder)
print(f'score : {score}')

In [ ]:
import matplotlib.pyplot as plt
% matplotlib inline

import seaborn as sns

# Use plot styling from seaborn.
sns.set(style='darkgrid')

# Increase the plot size and font size.
sns.set(font_scale=1.5)
plt.rcParams["figure.figsize"] = (12,6)

# Plot the learning curve.
plt.plot(df_stats['Training Loss'], 'b-o', label="Training")

# Label the plot.
plt.title("Training Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.xticks([1, 2, 3])

plt.show()